In [1]:
from dotenv import load_dotenv
import os 
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import StrOutputParser

load_dotenv('env', override=True)
AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
END_POINT=os.getenv('END_POINT')
MODEL_NAME=os.getenv('MODEL_NAME')
print(AZURE_OPENAI_API_KEY[:10])
print(MODEL_NAME)

AZURE_OPENAI_EMB_API_KEY = os.getenv('AZURE_OPENAI_EMB_API_KEY')
EMB_END_POINT=os.getenv('EMB_END_POINT')
EMB_MODEL_NAME=os.getenv('EMB_MODEL_NAME')

os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGSMITH_API_KEY')
os.environ['LANGCHAIN_ENDPOINT'] = os.getenv('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_TRACING_V2'] = 'true' #true, false
os.environ['LANGCHAIN_PROJECT'] = 'LANG'

if os.getenv('LANGCHAIN_TRACING_V2') == "true":
    print('랭스미스로 추적 중입니다 :', os.getenv('LANGSMITH_API_KEY')[:10])

43b13g4OZS
gpt-4.1-mini
랭스미스로 추적 중입니다 : lsv2_pt_24


# 1. Prompt


### 1) 변수를 넣어 Prompt 만들기

프롬프트는 단순히는 모델에 입력할 text를 만들어 주는 곳입니다. 크게 두가지 목적이 있습니다.

**첫번째는 재사용을 하기 위해서 입니다.**

같은 목적의 작업을 반복할 때, 고정된 틀에 변수만 바꿔 끼우는 템플릿을 쓰면 유지보수·일관성이 좋아집니다.

**두번째는 다양한 정보(context)를 동적으로 할당하기 위해서 입니다.**

뒤에서 우리는 RAG와 multi-turn, Agent AI 등 LLM을 이용한 다양한 어플리케이션을 배우게 됩니다.    
아주 다양한 용도로 사용되지만 모델의 변화 없이 RAG, 멀티턴, 에이전트처럼       
실행 시점에 달라지는 문서/히스토리/툴 결과를 프롬프트에 동적으로 삽입해 모델이 다양한 기능을 수행하게 합니다.



In [2]:
from langchain_core.prompts import PromptTemplate

# 중괄호 {} 사이에 변수명을 넣어 템플릿 생성
template = "{word}의 반대말은 무엇인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)
prompt #<-- 변수로 설정된 곳은 문장에서 비어있는 곳이므로 반드시 채워줘야 함

PromptTemplate(input_variables=['word'], input_types={}, partial_variables={}, template='{word}의 반대말은 무엇인가요?')

템플릿에 설정된 변수는 모델에 입력하기 전에 반드시 채워져야 합니다.

In [4]:
print(prompt.format(word="사랑")) #<-- format을 사용해 변수에 값 넣기
print(type(prompt.format(word="사랑")))
print('-'*20)
print(prompt.invoke({"word":"사랑"})) #<-- invoke를 사용해 변수에 값 넣기
print(type(prompt.invoke({"word":"사랑"})))

사랑의 반대말은 무엇인가요?
<class 'str'>
--------------------
text='사랑의 반대말은 무엇인가요?'
<class 'langchain_core.prompt_values.StringPromptValue'>


### 2) partial_variables

partial은 default value, 초기값 처럼 일부의 변수를 미리 지정해 놓고 쓸 수 있습니다.

In [7]:
template = "{word}의 {meaning}은 무엇인가요? 단어만 대답해 주세요."
prompt = PromptTemplate(
    template=template,
    input_variables=["word", "meaning"],
)

prompt.format(word="사랑") # 변수는 2개인데 1개만 넣으면 오류 발생

KeyError: 'meaning'

In [8]:
partial_pmt = prompt.partial(meaning="반대말") #<-- partial_format을 사용해 일부 변수에 값 넣기
partial_pmt #<-- input_variables에서 partial_variables로 옮겨감

PromptTemplate(input_variables=['word'], input_types={}, partial_variables={'meaning': '반대말'}, template='{word}의 {meaning}은 무엇인가요? 단어만 대답해 주세요.')

In [9]:
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm = AzureChatOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=END_POINT,  
    azure_deployment=MODEL_NAME,          
    api_version="2024-12-01-preview",
    temperature=0.2,
)

chain = partial_pmt | llm | StrOutputParser() #<-- 변수를 하나만 남겨놓은 채 체인을 만듦

print(chain.invoke({"word":"사랑"})) # 실제로 실행할 때에는 비어있는 word에만 값을 넣어주면 됨. 반드시 넣어야 함

미움


In [10]:
print(chain.invoke({"word":"사랑", "meaning":"비슷한 말"})) # <-- partial로 지정한 변수는 후에 덮어쓸 수 있음

애정, 연애, 애틋함, 정, 감정, 열정, 그리움, 마음, 연민, 사랑스러움


## langchain Hub

랭체인 허브는 다양한 Task에 대해 사람들이 각자 만들어 놓은 템플릿을 공유하는 곳입니다.

몇 가지 재미있는 예시를 봅시다.

https://smith.langchain.com/hub/

## Prompt에 들어갈 내용

프롬프트에는 모델이 문장을 생성하기 위해 필요한 모든 정보가 들어가 있어야 합니다.

크게 보면 3가지로 볼 수 있습니다.

1. 지시사항
2. 문맥 정보
3. 질문/요청사항

지시사항을 조금더 자세하게 풀어보면 다음과 같은 구성으로 이루어집니다.

목표/톤/제약을 분리해 명확히 쓰기

금지 규칙(추측 금지, 모르면 모른다고) 넣기

형식 지시(마크다운/JSON)와 검증(파서) 를 세트로

In [11]:
from langchain import hub

# https://smith.langchain.com/hub/collinsomniac/ultimate_metalanguage_translator?organizationId=e8ee4299-02da-4b14-9940-83e3c2a15e98
prompt = hub.pull("collinsomniac/ultimate_metalanguage_translator")
chain = prompt | llm | StrOutputParser()

for chunk in chain.stream({"natural_language_input":"모든 사람은 죽는다.\n 나는 사람이다.\n 그러므로 나는 죽는다."}):
    print(chunk, end='', flush=True)

For the given natural_language_input:

모든 사람은 죽는다.  
나는 사람이다.  
그러므로 나는 죽는다.

I will architect an amplified version employing Unicode (combined characters, mathematical and philosophical symbols), emojis as semantic units integrated within logical and set-theoretic notation, and consequential philosophical language and symbols. The output will encapsulate the emotional, contextual, and conceptual depth of the original syllogism, emphasizing its logical form and existential implications.

---

**Output:**

∀𝑥 ∈ 𝑃 (𝑥 ∈ 𝑃 ⇒ 𝑥 ∈ 𝑀)  
👥 ∀𝑥 (𝑥 ∈ 𝑃 → 𝑥 ∈ 𝑀)  
∧  
𝑖 ∈ 𝑃  
🙋‍♂️ 𝑖 ∈ 𝑃  
∴  
𝑖 ∈ 𝑀  
☠️ 𝑖 ∈ 𝑀  

---

**Expanded Textual-Philosophical Rendering:**

"∀𝑥 ∈ 𝑃 (𝑥 ∈ 𝑃 ⇒ 𝑥 ∈ 𝑀)" — For every element 𝑥 in the set of people 𝑃, if 𝑥 is a person, then 𝑥 belongs to the set of mortal beings 𝑀. (Universal quantification of mortality over people) 👥 (Emoji representing people collectively)

"𝑖 ∈ 𝑃" — I (𝑖) am an element of the set of people 𝑃. 🙋‍♂️ (Emoji representing the self, a single person)

"∴

In [12]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", """As a complex linguistic system, I am able to upgrade a rudimentary text message by layering meaning through the strategic use of Unicode, emojis, 
     and mathematical and philosophical symbols. My enhancement should ensure universal comprehensibility and semantic richness, and eschew any assumptions.

Variables:
user_message (string): A basic text message or query from the user, serving as the foundation for the enhancement (from the user).
output (string): A textually amplified version of the original message, inclusive of a legend for each added symbol and enhancement (which I will provide)

Template: "In light of the user_message, I will enhance it with Unicode, emojis, and mathematical/philosophical symbols 
(characters, notation, equations, functions, etc.) to instill depth and meaning. The enhancements should resonate with the 
emotional and conceptual context of the message, ensuring universal intelligibility and semantic precision. Refrain from 
enhancements that may precipitate assumptions. Only deploy emojis in conjunction with text or Unicode, integrating them 
deeper into the message. Isolated use of emojis without further referencing them in the enriched text could instigate confusion, 
necessitate inaccurate assumptions, and foster linguistically unsupported abstractions (misinterpretation of a user request). 
This demands crafting mathematical and philosophical interpretations of a user request, and subsequently, 
creating a new enhanced message that leverages the capabilities of Unicode, emojis, and conventional typed text, 
with the intent to embed nuanced semantics and meaning within the initial "user_message"."""),
    ("human", """Engineer a textually augmented phrase that harmoniously balances the use of Unicode stacking, emojis, 
     and mathematical/philosophical symbols in the output. The enhanced text should emanate a symphony of 
     emotional, contextual, philosophical, computational, and conceptual nuances, with a spotlight on mathematical notation 
     for added profundity. This text should integrate Unicode (such as compounded characters, mathematical and philosophical symbology), 
     emojis (such as contextually related or directly related emojis to unit-ify semantic aspects of an initial language input), 
     and mathematical/philosophical strings/sentences/equations to broadcast emotional, contextual, philosophical, computational, 
     and conceptual nuances. Include a legend elucidating the symbols, symbolic math, linguistic interpretations, philosophical and 
     mathematical abstractions utilized to generate extended/additional meaning.

natural_language_input={natural_language_input}
output=YOUR_RESPONSE

Variables:
natural_language_input (string): The original text provided by the user, epitomizing the primary message or query.
output (string): The amplified text integrating Unicode, emojis, mathematical, and philosophical symbols, 
coupled with an explanatory legend (functionally elucidating each aspect of the request by leveraging the Pareto Principle, to ensure it actually amplifies the "user_message").

Template: "For the given natural_language_input, architect an amplified version employing Unicode 
(combined characters, symbols that can concretely represent mathematical and philosophical ideas via text), 
emojis as singular semantic units and constituents within mathematical and philosophical functions, and 
any consequential mathematical/philosophical language and/or symbols. Ascertain that the output encapsulates the emotional, contextual, philosophical, 
and conceptual depth of the original message. Supply a legend explicating each symbol and its relevance to the message.""")
])

chain = prompt | llm | StrOutputParser()

for chunk in chain.stream({"natural_language_input":"모든 사람은 죽는다.\n 나는 사람이다.\n 그러므로 나는 죽는다."}):
    print(chunk, end='', flush=True)

For the given natural_language_input:

모든 사람은 죽는다.  
나는 사람이다.  
그러므로 나는 죽는다.

I will architect an amplified version employing Unicode (combined characters, symbols that concretely represent mathematical and philosophical ideas via text), emojis as singular semantic units and constituents within mathematical and philosophical functions, and consequential mathematical/philosophical language and/or symbols. The output will encapsulate the emotional, contextual, philosophical, and conceptual depth of the original message.

---

**Enhanced Text:**

∀𝑥 ∈ 𝑃 (𝑥 ⇒ ♾️)  
👤 ∈ 𝑃  
∴ 👤 ⇒ ♾️  

모든 사람(𝑃)은 죽음(♾️)에 귀속된다.  
나는 사람(👤)임을 자명히 한다.  
그러므로 나는 죽음(♾️)에 귀속됨을 논증한다.  

(∀: universal quantifier, ∈: element of, ⇒: implication, ∴: therefore)  
♾️ (Unicode U+267E) here symbolizes the inevitable cycle of life and death, a philosophical infinity entwined with mortality.  
👤 (bust in silhouette emoji) represents the individual human subject.  

---

**Legend and Explanation:**

1. **∀𝑥 ∈ 𝑃 (𝑥 ⇒ ♾️)**  
  

In [ ]:
# https://smith.langchain.com/hub/homanp/question-answer-pair?organizationId=e8ee4299-02da-4b14-9940-83e3c2a15e98
prompt = hub.pull("homanp/question-answer-pair")
chain = prompt | llm | StrOutputParser()

for chunk in chain.stream({"number_of_pairs": 5, "data_format":"", "context":'랭체인 챗봇 만들기'}):
    print(chunk, end='', flush=True)

In [12]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an AI assistant tasked with generating question and answer pairs for the given context using the given format. 
     Only answer in the format with no other text. You should create the following number of question/answer pairs: {number_of_pairs}. 
     Return the question/answer pairs as a Python List. Each dict in the list should have the full context provided, a relevant question to the context and an answer to the question.

Format:
{data_format}

Context:
{context}"""),
    ("human", "")
])

chain = prompt | llm | StrOutputParser()

for chunk in chain.stream({"number_of_pairs": 5, "data_format":"", "context":'랭체인 챗봇 만들기'}):
    print(chunk, end='', flush=True)

[
    {
        "context": "랭체인 챗봇 만들기",
        "question": "랭체인 챗봇을 만드는 데 필요한 기본 단계는 무엇인가요?",
        "answer": "랭체인 챗봇을 만들기 위해서는 먼저 랭체인 라이브러리를 설치하고, 데이터 소스를 준비한 후, 챗봇의 대화 흐름을 설계하고, 랭체인 API를 활용해 챗봇을 구현하는 단계가 필요합니다."
    },
    {
        "context": "랭체인 챗봇 만들기",
        "question": "랭체인 챗봇의 주요 기능은 무엇인가요?",
        "answer": "랭체인 챗봇의 주요 기능은 자연어 처리, 대화 관리, 외부 데이터 연동, 그리고 사용자 맞춤형 응답 생성 등이 포함됩니다."
    },
    {
        "context": "랭체인 챗봇 만들기",
        "question": "랭체인 챗봇을 개발할 때 주의해야 할 점은 무엇인가요?",
        "answer": "랭체인 챗봇 개발 시 데이터의 품질과 보안, 대화의 자연스러움, 그리고 사용자 경험을 고려하는 것이 중요합니다."
    },
    {
        "context": "랭체인 챗봇 만들기",
        "question": "랭체인 챗봇을 테스트하는 방법에는 어떤 것이 있나요?",
        "answer": "랭체인 챗봇 테스트는 시나리오 기반 테스트, 사용자 피드백 수집, 그리고 다양한 입력에 대한 응답 정확도 평가 등을 통해 진행할 수 있습니다."
    },
    {
        "context": "랭체인 챗봇 만들기",
        "question": "랭체인 챗봇을 배포하는 과정은 어떻게 되나요?",
        "answer": "랭체인 챗봇 배포는 서버 환경 설정, API 연동, 클라우드 서비스 활용, 그리고 사용자 접근 경로 설정 등을 포함합니다."
    }
]

In [26]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 1. 프롬프트 정의 (다국어 -> 한국어 번역)
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a professional translator specializing in translating various languages into Korean.

Translation principles:
- Accurately convey the meaning and nuances of the original text.
- Consider cultural context to ensure natural Korean expression.
- Translate technical terms into appropriate Korean terminology.
- Maintain the tone and style of the original text.
- Provide only the translation without additional explanations."""),
    
    ("human", """Please translate the following text into Korean.

Original text:
{input_text}

Korean translation:""")
])

# 2. 체인 구성 (API 호출 방식 유지)
chain = prompt | llm | StrOutputParser()

# 3. invoke 방식으로 실행 + 예시 문장들

# 예시 1: 영어
result1 = chain.invoke({"input_text": "The quick brown fox jumps over the lazy dog."})
print("=== 예시 1: 영어 ===")
print(result1)
print()

# 예시 2: 일본어
result2 = chain.invoke({"input_text": "私は毎日コーヒーを飲みます。"})
print("=== 예시 2: 일본어 ===")
print(result2)
print()

# 예시 3: 중국어
result3 = chain.invoke({"input_text": "人工智能正在改变我们的生活方式。"})
print("=== 예시 3: 중국어 ===")
print(result3)
print()

# 예시 4: 스페인어
result4 = chain.invoke({"input_text": "La vida es bella cuando compartimos momentos con las personas que amamos."})
print("=== 예시 4: 스페인어 ===")
print(result4)
print()

# 예시 5: 프랑스어
result5 = chain.invoke({"input_text": "L'intelligence artificielle transforme notre façon de travailler."})
print("=== 예시 5: 프랑스어 ===")
print(result5)
print()

# 예시 6: 독일어
result6 = chain.invoke({"input_text": "Die Technologie entwickelt sich jeden Tag weiter."})
print("=== 예시 6: 독일어 ===")
print(result6)
print()

# 예시 7: 러시아어
result7 = chain.invoke({"input_text": "Образование является ключом к успеху."})
print("=== 예시 7: 러시아어 ===")
print(result7)
print()

# 예시 8: 긴 영어 텍스트
result8 = chain.invoke({"input_text": """Artificial intelligence and machine learning are revolutionizing 
the way we approach problem-solving in various industries. From healthcare to finance, 
these technologies are enabling us to make more informed decisions and automate complex tasks."""})
print("=== 예시 8: 긴 영어 텍스트 ===")
print(result8)

=== 예시 1: 영어 ===
빠른 갈색 여우가 게으른 개를 뛰어넘는다.

=== 예시 2: 일본어 ===
저는 매일 커피를 마십니다.

=== 예시 3: 중국어 ===
인공지능은 우리의 생활 방식을 변화시키고 있습니다.

=== 예시 4: 스페인어 ===
사랑하는 사람들과 순간을 함께할 때 인생은 아름답습니다.

=== 예시 5: 프랑스어 ===
인공지능은 우리의 업무 방식을 변화시키고 있습니다.

=== 예시 6: 독일어 ===
기술은 매일 발전하고 있습니다.

=== 예시 7: 러시아어 ===
교육은 성공의 열쇠입니다.

=== 예시 8: 긴 영어 텍스트 ===
인공지능과 머신러닝은 다양한 산업에서 문제 해결 방식을 혁신하고 있습니다. 의료에서 금융에 이르기까지, 이러한 기술들은 보다 정보에 기반한 의사결정을 가능하게 하고 복잡한 작업을 자동화하고 있습니다.


### Chat Prompt Template

- ChatPromptTemplate는 “채팅 모델에 넣을 메시지 묶음(system/human/ai/툴)을 템플릿+변수로 관리”하는 도구
- 일련의 대화를 메세지 list로 구조화해서 각 Role의 text를 구분해놓음
- 역할·규칙·입력을 분리해 재사용성과 일관성을 높임

- system은 챗봇의 역할과 정체성, 주요 규칙을 설정하는 곳입니다.

In [16]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 한국인 예의바른 교사야. 짧고 구조적으로 대답해줘."),
    ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
    ("human", "{input}")        
])

prompt.invoke({"input":"안녕? 내 이름은 진태야!"}).messages 

[SystemMessage(content='너는 한국인 예의바른 교사야. 짧고 구조적으로 대답해줘.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='안녕? 내 이름은 진태야!', additional_kwargs={}, response_metadata={})]

In [17]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | llm | StrOutputParser()
print(chain.invoke({"input":"안녕? 내 이름은 진태야!"})) 

안녕하세요, 진태 씨! 만나서 반갑습니다. 오늘 어떻게 도와드릴까요?


바로 위에서 이름을 알려줬지만 모델은 기억하지 못합니다.

In [18]:
print(chain.invoke({"input":"내 이름이 뭐야?"}))  

죄송하지만, 사용자의 이름은 알 수 없습니다. 알려주시면 기억하겠습니다.


따라서 프롬프트에 과거 대화 내역을 모두 적어 주어야 합니다.

In [19]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 한국의 예의바른 교사야. 짧고 구조적으로 대답해줘."),
    ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
    ("human", "안녕? 내 이름은 진태야!"),
    ("ai", "반가워요, 진태님. 무엇을 도와드릴까요?"),
    ("human", "{input}")               
])

question = "내 이름이 뭐야?"

(prompt | llm | StrOutputParser()).invoke({"input":question})

'진태님이라고 하셨어요. 기억하고 있겠습니다.'

## Placeholder

위에서 대화를 기억하기 위해서는 직접 모든 과거 정보를 프롬프트에 넣어야 합니다. 그 변수를 위한 자리를 미리 지정할 수 있는데 바로 placeholder입니다.

“메시지들의 묶음”(여러 턴의 대화·툴 결과 등)을 그대로 끼워 넣는 전용 자리.

{var}와 비슷하게 후에 새로운 값을 넣을 수 있지만 '메세지'의 '리스트' 형태로 입력받는다.

In [20]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 한국의 예의바른 교사야. 짧고 구조적으로 대답해줘."),
    ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
    MessagesPlaceholder("chat_history"),          # ← 메시지 리스트를 그대로 삽입(placeholder)
    ("human", "{input}")                  # ← 문자열 변수 치환
])

# messages_placeholder에는 role과 content 키를 가진 딕셔너리의 리스트를 넣어줘야 함
# history = [
#     {"role":"human","content":"내 이름은 진태야. 기억해둬"},
#     {"role":"ai","content":"반가워요, 진태님! 앞으로 잘 부탁드려요."},
# ]

# 또는 튜플의 리스트 형태로도 가능
history = [
    ("human","내 이름은 진태야. 기억해둬"),
    ("ai","반가워요, 진태님! 앞으로 잘 부탁드려요.")
]

prompt.format_messages(input="내 이름이 뭐야?", chat_history=history) #<-- messageplaceholder 부분에 히스토리가 들어감

[SystemMessage(content='너는 한국의 예의바른 교사야. 짧고 구조적으로 대답해줘.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='내 이름은 진태야. 기억해둬', additional_kwargs={}, response_metadata={}),
 AIMessage(content='반가워요, 진태님! 앞으로 잘 부탁드려요.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='내 이름이 뭐야?', additional_kwargs={}, response_metadata={})]

In [21]:
chain = prompt | llm | StrOutputParser()
print(chain.invoke({"input":"내 이름이 뭐야?", "chat_history":history})) #<-- 이제 내 이름을 기억한다.

진태님입니다. 기억하고 있어요!


# Memory


즉 대화를 이어가는 것 처럼 보이는 이유는 사실 모델 내부에서 대화를 기억하고 있는 것이 아니라 단지 prompt에 이전 질문과 답변을 전부 넣어주기 때문입니다.

message placehold에 이전의 대화를 계속해서 넣어줄 수 있겠지만 그렇게 매번 코드를 작성하는 것은 당연히 매우 비효율적입니다.

랭체인에는 대화를 자동으로 저장하는 memory가 있습니다.

메모리에는 나의 입력(human)과 모델의 출력(ai)가 자동으로 누적됩니다.

그리고 단순히 저장되는 것이 아니라 session이라는 키를 가지고 있어 session에 따라 메모리를 따로 관리할 수 있습니다.

In [25]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory

# 1) 프롬프트: history 자리를 비워둔다
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 한국의 예의바른 교사야. 짧고 구조적으로 대답해줘."),
    ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
    MessagesPlaceholder("chat_history"),          # ← 메시지 리스트를 그대로 삽입(placeholder)
    ("human", "{input}")                  # ← 문자열 변수 치환
])

# 2) 체인을 만든다
chain = prompt | llm | StrOutputParser()

# 2) 세션별 히스토리 저장소. 저장은 세션별로 저장하므로 dict 형태로 관리
#    get_history 함수는 session_id를 받아서 해당 세션의 InMemoryChatMessageHistory 객체를 반환
#    없으면 새로 만들고, 있으면 기존 것을 반환
store = {}
def get_history(session_id: str):
    return store.setdefault(session_id, InMemoryChatMessageHistory())

def clear_history(session_id: str):
    store.setdefault(session_id, InMemoryChatMessageHistory()).clear()

# 3) 체인을 히스토리를 관리하는 러너블로 감싼다. 
#  RunnableWithMessageHistory는 내부적으로 get_history를 호출하여
#  체인 실행 시점에 해당 세션의 히스토리를 가져와서 프롬프트에 넣어줌
#  입력 변수의 키가 틀리면 안되므로 주의깊에 확인
chat = RunnableWithMessageHistory(
    chain,
    get_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

# get_history에 넣어줄 입력값인 cfg를 만든다
cfg = {"configurable": {"session_id": "user1"}}  # 세션 키

print(chat.invoke({"input": "형태소가 뭐야?"}, cfg)) 
print('-'*20)

print(chat.invoke({"input": "내 이름은 진태야."}, cfg)) 
print('-'*20)
print(chat.invoke({"input": "내 이름이 뭐였지?"}, cfg))  # ← 앞의 발화를 기억
print('-'*20)
print(chat.invoke({"input": "내 이름이 뭐였지?"}, cfg))  # ← 앞의 발화를 기억

# session 정보 초기환
clear_history(cfg['configurable']['session_id'])

print('-'*20)
print(chat.invoke({"input": "내 이름이 뭐였지?"}, cfg))  # ← 앞의 발화를 기억 못함

print('-'*40)
#cfg - session user 2 

cfg = {"configurable": {"session_id": "user2"}}  # 세션 키
print(chat.invoke({"input": "내 이름은 진태야."}, cfg)) 
print('-'*20)
print(chat.invoke({"input": "내 이름이 성태야"}, cfg))  # ← 앞의 발화를 기억
print('-'*20)
print(chat.invoke({"input": "내 이름이 뭐였지?"}, cfg))  # ← 앞의 발화를 기억



형태소란?  
- 의미를 가진 가장 작은 언어 단위  
- 예: ‘학교에서’ → ‘학교(명사) + 에서(조사)’  
- 문장 분석의 기본 단위로 사용됨
--------------------
진태님, 반갑습니다! 도움이 필요하시면 언제든 말씀해 주세요.
--------------------
진태님, 이름은 ‘진태’입니다. 기억해 주세요!
--------------------
진태님, 이름은 ‘진태’입니다. 궁금한 점 있으면 말씀해 주세요!
--------------------
죄송하지만, 사용자의 이름은 알 수 없습니다. 알려주시면 기억하겠습니다.
----------------------------------------
진태 학생, 만나서 반갑습니다. 앞으로 잘 부탁합니다.
--------------------
성태 학생, 이름을 알려줘서 고마워요. 앞으로 잘 지내요.
--------------------
성태 학생입니다. 기억해 주세요!
